In [6]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords # stop words use to not include a , an or such things which is not much useful
from nltk.stem.porter import PorterStemmer#removes prefix and sufix
from sklearn.feature_extraction.text import TfidfVectorizer #convert text to different features
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score # for logistic Regression accuracy score works


In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
df = pd.read_csv("train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# **Preprocessing the dataset**

In [20]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


Dropping the missing values because of very large dataset

In [17]:
df.dropna(axis=1,inplace=True)

In [18]:
df

,id,label
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
20795,20795,0
20796,20796,0
20797,20797,0
20798,20798,1


In [24]:
df.shape

(20800, 5)

In [25]:
df.dropna(axis=0,inplace=True)

In [26]:
df.tail(5)

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [27]:
df.drop('id',axis=1,inplace=True)

In [30]:
df['content'] = df['author'] +" "+ df['title']

In [32]:
df.drop(['title','author'],axis=1,inplace=True)

In [33]:
df

,text,label,content
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...
20795,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [34]:
X = df.drop(['label'],axis=1)
y = df['label']

# **Stemming**

basically removes the prefix and sufix
it is a process of reducing a word into it's root word.
There can be several words:
actors , acting , act

In [36]:
port_stemmer = PorterStemmer()

In [37]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #removing numbers except for alphabets
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english') ]
  stemmed_content = " ".join(stemmed_content)
  return stemmed_content

In [39]:
df['content'] =  df['content'].apply(lambda x:stemming(x))

In [45]:
df

,label,content
0,1,darrel lucu hous dem aid even see comey letter...
1,0,daniel j flynn flynn hillari clinton big woman...
2,1,consortiumnew com truth might get fire
3,1,jessica purkiss civilian kill singl us airstri...
4,1,howard portnoy iranian woman jail fiction unpu...
...,...,...
20795,0,jerom hudson rapper trump poster child white s...
20796,0,benjamin hoffman n f l playoff schedul matchup...
20797,0,michael j de la merc rachel abram maci said re...
20798,1,alex ansari nato russia hold parallel exercis ...


In [44]:
df.drop('text',axis=1,inplace=True)

In [101]:
X = df.drop('label', axis=1).values
y = df['label'].values

In [102]:
print(y)

[1 0 1 ... 0 1 1]


In [103]:
print(X)

[['darrel lucu hous dem aid even see comey letter jason chaffetz tweet']
 ['daniel j flynn flynn hillari clinton big woman campu breitbart']
 ['consortiumnew com truth might get fire']
 ...
 ['michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time']
 ['alex ansari nato russia hold parallel exercis balkan']
 ['david swanson keep f aliv']]


In [104]:
X_flatten  = [sent[0] for sent in X]

In [110]:
X = X_flatten.copy()

In [109]:
type(X_flatten)

list

In [89]:
print(X.shape,y.shape)

(18285, 1) (18285,)


In [111]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet', 'daniel j flynn flynn hillari clinton big woman campu breitbart', 'consortiumnew com truth might get fire', 'jessica purkiss civilian kill singl us airstrik identifi', 'howard portnoy iranian woman jail fiction unpublish stori woman stone death adulteri', 'daniel nussbaum jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart', 'alissa j rubin beno hamon win french socialist parti presidenti nomin new york time', 'megan twohey scott shane back channel plan ukrain russia courtesi trump associ new york time', 'aaron klein obama organ action partner soro link indivis disrupt trump agenda', 'chri tomlinson bbc comedi sketch real housew isi caus outrag', 'amando flavio russian research discov secret nazi militari base treasur hunter arctic photo', 'jason ditz us offici see link trump russia', 'anotheranni ye paid govern troll social media blog forum websit', 'jack william major

In [114]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [115]:
print(X)

  (0, 247)	0.26982554594264346
  (0, 2312)	0.3745612250433202
  (0, 2757)	0.2466340295002162
  (0, 3359)	0.3609049070394367
  (0, 3543)	0.2684494960336511
  (0, 4637)	0.23016077319140021
  (0, 6552)	0.21745594418933306
  (0, 7190)	0.24556189342497173
  (0, 8048)	0.29347549279156676
  (0, 8310)	0.3609049070394367
  (0, 12567)	0.25566372256502734
  (0, 14626)	0.2853880981846006
  (1, 1391)	0.2961798071396214
  (1, 1764)	0.15099851642776987
  (1, 2066)	0.3819189043603919
  (1, 2619)	0.1936832753563371
  (1, 3328)	0.2623789770430963
  (1, 5140)	0.7119376870709987
  (1, 6377)	0.19285723710368194
  (1, 15663)	0.30530279633389806
  (2, 2743)	0.31437590974242646
  (2, 2895)	0.4581003415623782
  (2, 5031)	0.38709995799949964
  (2, 5579)	0.3490632212946542
  (2, 8973)	0.4948460479407663
  :	:
  (18282, 3399)	0.21709587881547218
  (18282, 6585)	0.21510476155370073
  (18282, 7798)	0.2268797993396579
  (18282, 8382)	0.35970450365294165
  (18282, 8879)	0.29296479379736134
  (18282, 8942)	0.171295501

# **Splitting the dataset**

In [116]:
X_train,X_test,y_train,y_test  = train_test_split(X,y,test_size=0.33,random_state=42)

# **Logistic Regression Model**

In [117]:
model = LogisticRegression()

In [120]:
model.fit(X_train,y_train)

LogisticRegression()

Predicting

In [122]:
pred = model.predict(X_test)
#accuracy score analysing
print(accuracy_score(y_test,pred))

0.9792874896437448


In [124]:
X_new = X_test[0]
prediction = model.predict(X_new)
if prediction==0:
  print("The news is fake")
else:
  print("The news is real!")

The news is real!


In [125]:
print(y_test[0])

1
